<a href="https://colab.research.google.com/github/saotomryo/Use_MMClassification/blob/main/Use_MMClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import json
from skimage import measure
import numpy as np
import os
from glob import glob
import pandas as pd
from sklearn.model_selection import train_test_split
print(torch.__version__)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device = ", device)

## 環境準備 MMCV MMDetectionのインストール

In [ ]:

# MMCVのインストール
!pip install -U openmim
!mim install mmcv-full

In [ ]:
HOME_PATH = "/content"

In [ ]:

!git clone https://github.com/open-mmlab/mmclassification.git
%cd mmclassification
!pip install -r requirements.txt
!pip install "git+https://github.com/open-mmlab/cocoapi.git#subdirectory=pycocotools"
!pip install -v -e .  # or "python setup.py develop"

## コンフィグファイルの編集

In [ ]:
from mmcv import Config
cfg = Config.fromfile('/content/mmclassification/configs/swin_transformer/swin-base_16xb64_in1k-384px.py')


In [ ]:
print(f'Config:\n{cfg.pretty_text}')

In [ ]:
%cd /content/

/content


##MMGenerationでの学習データは以下のような形式でテキストファイルを作成してください。


1 画像ファイル名 正解ラベル（数字）  
2 画像ファイル名　正解ラベル（数字）  
3 .....  
.  
.  



In [ ]:
# コンフィグを変更する
# swin_transforemer

#data_root = '画像データのルートフォルダ'
data_root = '/content/drive/MyDrive/signate/Tanacho/train/'

cfg.model.head.num_classes = #分類するクラス数
cfg.data.test.data_prefix = # テストデータのパス
cfg.data.val.data_prefix = # 検証データのパス
cfg.data.train.data_prefix = # 学習データのパス

cfg.data.train.ann_file = # 学習用のアノテーションファイルのパス
cfg.data.test.ann_file = # テスト用のアノテーションファイルのパス
cfg.data.val.ann_file = # 検証用のアノテーションファイルのパス


# バッチサイズ
#cfg.data.samples_per_gpu = #メモリエラーが出る場合は、バッチサイズを減らしてみてください。
#cfg.data.workers_per_gpu = #メモリエラーが出る場合は、バッチサイズを減らしてみてください。


# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')


# 編集したコンフィグをファイル出力
cfg.dump('/content/swin_transformer.py')


In [ ]:
%cd "$HOME_PATH"
!mkdir -p "$HOME_PATH"/work_train
!python /content/mmclassification/tools/train.py \
    '/content/swin_transformer.py' \
    --work-dir "$HOME_PATH"/work_train


## 学習結果の確認

In [ ]:
from argparse import ArgumentParser
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
model = init_detector("/content/swin_transformer.py", "学習済みモデル", device=device)
result = inference_detector(model, "予測結果を確認する画像のパス")
